### Sandbox for exporting camera metadata from a database

In [10]:
from dataclasses import dataclass
from pathlib import Path

import polars as pl

from afft.io import read_config
from afft.io.sql import create_endpoint
from afft.utils.log import logger
from afft.utils.result import Ok, Err

from afft.tasks.process_tables import process_database_tables


def load_task_configs(path: Path) -> list[TaskConfig]:
    """Load a collection of task configurations from file."""
    match read_config(path):
        case Ok(data):
            return [TaskConfig(**config) for config in data.get("tasks")]
        case Err(message):
            logger.error(message)
            return list()


def handle_tasks(endpoint, configs: list[TaskConfig]) -> dict[str, pl.DataFrame]:
    """Execute joining of camera."""

    results: dict[str, pl.DataFrame] = dict()
    for config in configs:

        results[config.label] = process_database_tables(
            endpoint,
            queries=config.queries,
            selections=config.selections,
            base=config.join.get("base"),
            join_on=config.join.get("field"),
        )

    return results


def main() -> None:
    """Main function."""

    CONFIG: Path = Path("../config/message_queries/qd61g27j.toml")
    OUTPUT: Path = Path("/data/kingston_snv_01/acfr_cameras_metadata")

    configs: list[TaskConfig] = load_task_configs(CONFIG)

    logger.info("")
    for config in configs:
        logger.info(config.label)
    logger.info("")

    match create_endpoint(database="acfr_auv_messages", host="localhost", port=5432):
        case Ok(endpoint):
            results: dict[str, pl.DataFrame] = handle_tasks(endpoint, configs)

            for name, data in results.items():
                output: Path = OUTPUT / f"{name}_cameras_metadata.csv"
                error: str | None = data.write_csv(output)
                if not error:
                    logger.info(f"Saved data frame: {output}")
                else:
                    logger.error(f"Error when writing data: {error}")
        case Err(message):
            logger.error(message)


# INVOKE MAIN
main()

2024-10-16 17:08:21.535 | INFO     | __main__:main:59 - 
2024-10-16 17:08:21.536 | INFO     | __main__:main:61 - qd61g27j_20100421_022145
2024-10-16 17:08:21.536 | INFO     | __main__:main:61 - qd61g27j_20110410_011202
2024-10-16 17:08:21.536 | INFO     | __main__:main:61 - qd61g27j_20120422_043114
2024-10-16 17:08:21.537 | INFO     | __main__:main:61 - qd61g27j_20130414_013620
2024-10-16 17:08:21.537 | INFO     | __main__:main:61 - qd61g27j_20170523_040815
2024-10-16 17:08:21.537 | INFO     | __main__:main:62 - 
2024-10-16 17:08:22.646 | INFO     | __main__:main:72 - Saved data frame: /data/kingston_snv_01/acfr_cameras_metadata/qd61g27j_20100421_022145_cameras_metadata.csv
2024-10-16 17:08:22.648 | INFO     | __main__:main:72 - Saved data frame: /data/kingston_snv_01/acfr_cameras_metadata/qd61g27j_20110410_011202_cameras_metadata.csv
2024-10-16 17:08:22.649 | INFO     | __main__:main:72 - Saved data frame: /data/kingston_snv_01/acfr_cameras_metadata/qd61g27j_20120422_043114_cameras_me